# Generate TOY Tensor 

In [1]:
import random
import numpy as np

random.seed(42)
np.random.seed(42)

In [2]:
train_coords = list()
train_count = list()

test_coords = list()
test_count = list()

for ii in range(1000):
    train_coords.append(list(np.random.randint(10, size=4)))
    train_count.append(random.randint(1, 20))
    
for ii in range(100):
    test_coords.append(list(np.random.randint(10, size=4)))
    test_count.append(random.randint(1, 20))

In [3]:
tensor = dict()
tensor['train_coords'] = train_coords
tensor['train_count'] = train_count
tensor['test_coords'] = test_coords
tensor['test_count'] = test_count

In [4]:
np.savez_compressed('TOY',**tensor)

# Load the TOY Tensor

In [5]:
data = np.load("TOY" + str(".npz"), allow_pickle=True)

In [6]:
list(data)

['train_coords', 'train_count', 'test_coords', 'test_count']

In [7]:
# Training set
coords_train = data['train_coords']
nnz_train = data['train_count']

# Test set
coords_test = data['test_coords']
nnz_test = data['test_count']

In [8]:
coords_train[:10]

array([[6, 3, 7, 4],
       [6, 9, 2, 6],
       [7, 4, 3, 7],
       [7, 2, 5, 4],
       [1, 7, 5, 1],
       [4, 0, 9, 5],
       [8, 0, 9, 2],
       [6, 3, 8, 2],
       [4, 2, 6, 4],
       [8, 6, 1, 3]])

In [9]:
nnz_train[:10]

array([ 4,  1,  9,  8,  8,  5,  4, 18,  3, 19])

In [10]:
coords_test[:10]

array([[1, 4, 1, 1],
       [5, 9, 8, 3],
       [5, 6, 5, 7],
       [3, 0, 6, 2],
       [5, 1, 8, 0],
       [5, 5, 4, 4],
       [0, 9, 4, 1],
       [6, 2, 3, 2],
       [2, 3, 4, 0],
       [3, 7, 9, 6]])

In [11]:
nnz_test[:10]

array([10, 10, 19, 19, 16,  5, 15, 18, 16, 12])

# Factorize

In [12]:
from pyCP_APR import CP_APR

cp_apr = CP_APR(n_iters=10, random_state=42, verbose=200, 
                method='torch',
                device='gpu', 
                return_type='numpy', 
                device_num=0
               )

/home/maksim/pyCP_APR/pyCP_APR/numpy_backend/CP_APR.py:220: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
  if self.verbose != 0 (self.simple_verbose == False):
/opt/conda/envs/pyCP_APR/lib/python3.9/site-packages/torch/__init__.py:614: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:451.)
  _C._set_default_tensor_type(t)


Using NVIDIA TITAN RTX


In [13]:
factors = cp_apr.fit(coords=coords_train, values=nnz_train, rank=[1,5])
factors

/home/maksim/pyCP_APR/pyCP_APR/torch_backend/CP_APR_Torch.py:463: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.epsilon = tr.tensor(self.epsilon).to(self.device)


CP-APR (MU):
Iter=1, Inner Iter=8, KKT Violation=0.000000, obj=-9748.910692, nViolations=0
Exiting because all subproblems reached KKT tol.
 Final log-likelihood = -9748.910692
 Final least squares fit = 0.041186
 Final KKT violation = 0.000000
 Total inner iterations = 12
 Total execution time = 0.8494 seconds
Converting the latent factors to Numpy arrays.
CP-APR (MU):
Iter=1, Inner Iter=40, KKT Violation=0.277127, obj=-8929.931115, nViolations=0
 Final log-likelihood = -8017.186125
 Final least squares fit = 0.055189
 Final KKT violation = 0.160233
 Total inner iterations = 400
 Total execution time = 0.9528 seconds
Converting the latent factors to Numpy arrays.


[{'Factors': {'0': array([0.10385419, 0.10347446, 0.10005696, 0.09711411, 0.09407632,
          0.09379153, 0.11363205, 0.08800076, 0.11638504, 0.08961458]),
   '1': array([0.09056389, 0.10632238, 0.0995823 , 0.10091133, 0.11334726,
          0.09588001, 0.09720904, 0.11704955, 0.10015189, 0.07898234]),
   '2': array([0.10992975, 0.10613252, 0.09758876, 0.11866338, 0.09796848,
          0.09625973, 0.07898234, 0.09350674, 0.10176571, 0.09920258]),
   '3': array([0.10992975, 0.07442567, 0.11714448, 0.09103854, 0.11334726,
          0.09815834, 0.09711411, 0.10461363, 0.10727169, 0.08695652])},
  'Weights': array([10534.])},
 {'Factors': {'0': array([[1.31313342e-01, 7.88352893e-02, 1.05177421e-01, 1.66367367e-01,
           7.35723480e-03],
          [3.50352087e-02, 1.39263896e-01, 1.22471481e-01, 1.03049559e-01,
           1.88774236e-01],
          [4.94525703e-02, 1.40498071e-01, 1.14713223e-01, 2.23393507e-06,
           2.49321155e-01],
          [8.84527234e-02, 1.18779191e-01, 1

# Predict

In [14]:
y_score = cp_apr.predict_scores(coords=coords_test, values=nnz_test)
y_score

array([2.08910667e-11, 1.69627721e-07, 0.00000000e+00, 0.00000000e+00,
       7.77156117e-16, 3.11204214e-03, 1.44328993e-15, 3.10862447e-15,
       0.00000000e+00, 1.47644674e-10, 1.52042377e-07, 0.00000000e+00,
       2.83784107e-12, 1.08537623e-11, 1.09300347e-11, 2.39411829e-05,
       4.16389750e-06, 3.97624460e-06, 2.33146835e-15, 1.02362563e-13,
       1.78918854e-04, 3.96765953e-11, 3.10366945e-01, 1.58183483e-08,
       0.00000000e+00, 3.56175597e-07, 3.09356984e-11, 4.41464623e-04,
       2.46469511e-13, 4.88905154e-01, 1.93036049e-03, 0.00000000e+00,
       0.00000000e+00, 9.21087890e-07, 2.05273860e-01, 9.10382880e-15,
       1.01223411e-02, 2.77555756e-14, 1.11022302e-16, 7.97153696e-01,
       5.68406766e-03, 0.00000000e+00, 7.38477774e-02, 7.94507339e-02,
       0.00000000e+00, 9.39525513e-09, 5.57141300e-09, 0.00000000e+00,
       1.56122892e-11, 6.36792698e-02, 1.27120536e-13, 0.00000000e+00,
       1.02005033e-08, 3.10230812e-08, 1.44328993e-15, 0.00000000e+00,
      